# Create weights for SLF observations

### Add paths to run locally or off NIRD

In [1]:
import sys
# Add common resources folder to path
sys.path.append("/mnt/mcc-ns9600k/jonahks/git_repos/netcdf_analysis/Common/")
sys.path.append("/mnt/mcc-ns9600k/jonahks/git_repos/netcdf_analysis/")
sys.path.append("/mnt/mcc-ns9600k/jonahks/git_repos/netcdf_analysis/hdf_caliop/")
sys.path.append("/home/jonahks/git_repos/netcdf_analysis/")
sys.path.append("/home/jonahks/git_repos/netcdf_analysis/Common/")

from imports import (
    pd, np, xr, mpl, plt, sns, os, 
    datetime, sys, crt, gridspec,
    ccrs, metrics, Iterable
    )

from functions import (
    masked_average, add_weights, sp_map,
    season_mean, get_dpm, leap_year, share_ylims,
    to_png, mute_ax
    )

from classes import SatComp_Metric, CT_SLF_Metric
from collections import deque
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load feedback specific functions

In [2]:
from hdf_functions import *

### Check running location and adjust working directory appropriately.

In [3]:
host = os.uname()[1]
if 'jupyter' in host.split('-'): # Check if running on NIRD through the Jupyter Hub
    print('Running through MC2 Jupyter Hub')
    model_dir = '/mnt/mcc-ns9600k/jonahks/'
    os.chdir(model_dir)

else:  # Assume that we're running on a local machine and mounting NIRD
    print('Running on %s, attempting to mount ns9600k/jonahks/ from NIRD' % str(host))
    os.system('fusermount -zu ~/drivemount/')  # unmount first
    os.system('sshfs jonahks@login.nird.sigma2.no:"p/jonahks/" ~/drivemount/')    # Calling mountnird from .bashrc doesn't work
    os.chdir('/home/jonahks/drivemount/')
    save_dir = '~/DATAOUT/'
    save_to = os.path.expanduser(save_dir)

output_dir = 'figures/'
case_dir = 'satcomp/'   # inconsistent label compared to jupy_test
conv_dir ='convectivephase/'

# Check that each important directory can be accessed:    
access_paths = os.path.exists(output_dir) and os.path.exists(case_dir) and os.path.exists(conv_dir)
print('Can access all directory paths:', access_paths)

Running through MC2 Jupyter Hub
Can access all directory paths: True


In [4]:
cd caliop_slf_weight/

/mnt/mcc-ns9600k/jonahks/caliop_slf_weight


In [15]:
ls

CAL_LID_L3_Cloud_Occurrence-Standard-V1-00.2009-01A.hdf
CAL_LID_L3_Cloud_Occurrence-Standard-V1-00.2009-01A.nc
CAL_LID_L3_Cloud_Occurrence-Standard-V1-00.2009-02A.hdf
CAL_LID_L3_Cloud_Occurrence-Standard-V1-00.2009-02A.nc
CAL_LID_L3_Cloud_Occurrence-Standard-V1-00.2009-03A.hdf
CAL_LID_L3_Cloud_Occurrence-Standard-V1-00.2009-03A.nc
CAL_LID_L3_Cloud_Occurrence-Standard-V1-00.2009-04A.hdf
CAL_LID_L3_Cloud_Occurrence-Standard-V1-00.2009-04A.nc
CAL_LID_L3_Cloud_Occurrence-Standard-V1-00.2009-05A.hdf
CAL_LID_L3_Cloud_Occurrence-Standard-V1-00.2009-05A.nc
CAL_LID_L3_Cloud_Occurrence-Standard-V1-00.2009-06A.hdf
CAL_LID_L3_Cloud_Occurrence-Standard-V1-00.2009-06A.nc
CAL_LID_L3_Cloud_Occurrence-Standard-V1-00.2009-07A.hdf
CAL_LID_L3_Cloud_Occurrence-Standard-V1-00.2009-07A.nc
CAL_LID_L3_Cloud_Occurrence-Standard-V1-00.2009-08A.hdf
CAL_LID_L3_Cloud_Occurrence-Standard-V1-00.2009-08A.nc
CAL_LID_L3_Cloud_Occurrence-Standard-V1-00.2009-09A.hdf
CAL_LID_L3_Cloud_Occurrence-Standard-V1-00.2009-09A.nc
C

In [13]:
ftest = hdf_to_netcdf('CAL_LID_L3_Cloud_Occurrence-Standard-V1-00.2009-01A.hdf')

In [14]:
files = os.listdir('/mnt/mcc-ns9600k/jonahks/caliop_slf_weight')
os.chdir('/mnt/mcc-ns9600k/jonahks/caliop_slf_weight')

for file in files:
    if file[-4:] == '.hdf':
        out_path = file[:-4] + '.nc'
    #     print(out_path)

        _netcdf = hdf_to_netcdf(file)
        _netcdf.to_netcdf(out_path)
        _netcdf.close() # prevents the kernel from crashing :)